In [1]:
def classify_cloud(img):
    # add library
    import os.path
    import cv2
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import tensorflow as tf
    import seaborn as sns
    import PIL
    import pathlib
    
    from glob import glob
    from tensorflow.keras import layers
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report,accuracy_score
    from keras.models import load_model
    
    # 파일 경로 지정
    change_dir = os.chdir("C:/Users/hi/Desktop/DeepLearning/data/chatbot_dataset/cloud_data")
    test_dir = pathlib.Path(os.getcwd())
    real_img = list(test_dir.glob(img))
    PIL.Image.open(str(real_img[0]))
    
    img_height = 180
    img_width = 180
    
    class_names = ['고적운(altocumulus, Ac)', '고층운(altostratus, As)', '적란운(cumulonimbus, Cb)',
               '권적운(cirrocumulus, Cc)', '권운(cirrus, Ci)', '권층운(cirrostratus, Cs)', 
               '비행운(contrail, Ct)', '적운(cumulus, Cu)', '난층운(nimbostratus, Ns)',
               '층적운(stratocumulus, Sc)', '충운(stratus, St)'
               ]
    
    img = keras.preprocessing.image.load_img(
        real_img[0], target_size=(img_height, img_width)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    model2 = load_model('C:/Users/hi/Desktop/DeepLearning/data/cloud_model.h5')

    predictions = model2.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    return class_names[np.argmax(score)]
    print("{}으로 예측됨".format(class_names[np.argmax(score)]))

In [2]:
classify_cloud('real_img.jpg')

'난층운(nimbostratus, Ns)'

In [3]:
cloud = classify_cloud('real_img.jpg')
cloud

'난층운(nimbostratus, Ns)'

In [4]:
def find_out_weather(nx, ny):
    import requests
    import json
    import datetime
    
    vilage_weather_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?"
    service_key = "servicekey"
    
    today = datetime.datetime.today()
    base_date = today.strftime("%Y%m%d")
    base_time = "0800"
    
    payload = "serviceKey=" + service_key + "&" + "dataType=json" + "&" + "base_date=" + base_date + "&" + "base_time=" + base_time + "&" + "nx=" + str(nx) + "&" + "ny=" + str(ny)
    
    data = dict()
    data['date'] = base_date
    
    weather_data = dict()
    
    res = requests.get(vilage_weather_url + payload)
    items = res.json().get('response').get('body').get('items')
    
    for item in items['item']:
        
        # 기온
        if item['category'] == 'TMP':
            weather_data['tmp'] = item['fcstValue']

        # 기상상태
        if item['category'] == 'PTY':

            weather_code = item['fcstValue']

            if weather_code == '1':
                weather_state = '비'
            elif weather_code == '2':
                weather_state = '비/눈'
            elif weather_code == '3':
                weather_state = '눈'
            elif weather_code == '4':
                weather_state = '소나기'
            else:
                weather_state = '기상 이상 없음'

    weather_data['code'] = weather_code
    weather_data['state'] = weather_state
        
    data['weather'] = weather_data
    return data['date'], data['weather']['tmp'], data['weather']['state']

In [5]:
weather = find_out_weather(37, 128) # 8시에 측정된 날씨 출력
print(weather[0], ': 현재 기온은', weather[1], '도이며 현재 상태는', weather[2])

20220617 : 현재 기온은 19 도이며 현재 상태는 기상 이상 없음


In [6]:
def score():
    if '권운' in cloud:
        c_score = 0
    elif '고적운' in cloud:
        c_score = 0
    elif '적운' in cloud:
        c_score = 0
    elif '권적운' in cloud:
        c_score = 1
    elif '층운' in cloud:
        c_score = 1
    elif '고층운' in cloud:
        c_score = 2
    elif '난층운' in cloud:
        c_score = 2
    elif '층적운' in cloud:
        c_score = 2
    elif '적란운' in cloud:
        c_score = 2
    elif '권층운' in cloud:
        c_score = 3
    else:
        pass
    
    if '기상 이상 없음' in weather[2]:
        w_score = 0
    elif '비' in weather[2]:
        w_score = 1
    elif '눈' in weather[2]:
        w_score = 1
    elif '비/눈' in weather[2]:
        w_score = 1
    elif '소나기' in weather[2]:
        w_score = 1
    else:
        pass
    
    score = c_score + w_score
    
    if score == 0:
        score = '맑아요'
        cloth = ['pants', 'shorts', 'skirt']
    elif score == 1:
        score = '흐려요'
        cloth = ['blouse', 'dress', 'pants', 'shorts', 'skirt', 'Tshirt']
    elif score == 2:
        score = '비나 눈이 올 수 있어요. 우산을 챙겨 가세요'
        cloth = ['jacket', 'pants', 'Tshirt']
    elif score == 3:
        score = '비/눈이 많이 와요. 미끄러지지 않게 조심하세요'
        cloth = ['jacket', 'pants', 'sweater']
    else:
        score = '태풍이 와요.. 이런 날씨엔 집에 있어요'
        cloth = ['pants', 'sweater']
    
    return score, cloth

In [7]:
score()[0], score()[1]

('흐려요', ['blouse', 'dress', 'pants', 'shorts', 'skirt', 'Tshirt'])

In [8]:
def recommend_cloth(inputs):
    import os
    from PIL import Image
    import random
    path = 'C:/Users/hi/Desktop/DeepLearning/img-fashion/' # 의류 이미지 저장된 경로 입력
    fashion = os.listdir(path)
    fashion_list = [file for file in fashion if file.endswith('.jpg')]
    types = inputs
    kind = [s for s in fashion_list if types in s]
    random_img = random.sample(kind, 3)
    rand_img1 = Image.open(path+random_img[0])
    rand_img2 = Image.open(path+random_img[1])
    rand_img3 = Image.open(path+random_img[2])
    rand_img1 = rand_img1.resize((300, 400))
    rand_img2 = rand_img2.resize((300, 400))
    rand_img3 = rand_img3.resize((300, 400))
    rand_img1_size = rand_img1.size
    rand_img2_size = rand_img2.size
    rand_img3_size = rand_img3.size
    new_image = Image.new('RGB',(3*rand_img1_size[0], rand_img1_size[1]), (250,250,250))
    new_image.paste(rand_img1,(0,0))
    new_image.paste(rand_img2,(rand_img1_size[0],0))
    new_image.paste(rand_img3,(2*rand_img1_size[0],0))
    new_image.show() # 새 창에서 이미지 띄우기
    print('결과를 확인해주세요')

In [9]:
cloth_list = score()[1]
cloth_list[0]

'blouse'

In [10]:
for rc_cloth in cloth_list:
    recommend_cloth(rc_cloth)

결과를 확인해주세요
결과를 확인해주세요
결과를 확인해주세요
결과를 확인해주세요
결과를 확인해주세요
결과를 확인해주세요
